In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  
from librep.base.estimator import Estimator
%load_ext autoreload
%autoreload 2
from standartized_balanced import StandardizedBalancedDataset
import os,torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_data(dataset_name,sensors,normalize_data):    
    working_directory=f"result/{dataset_name}/"
    data_folder = f"../../../../../../experiment_executor/data/standartized_balanced/{dataset_name}/"
    dataset = StandardizedBalancedDataset(data_folder, sensors=sensors)
    X_train, y_train,X_test, y_test,X_val, y_val = dataset.get_all_data(normalize_data=normalize_data, resize_data=True)
    return X_train, y_train,X_test, y_test,X_val, y_val
        #print(f"shape: X_train {X_train.shape} --- X_test {X_test.shape} --- X_test {X_val.shape}")

/home/amparo/unicamp_hyper_2/M4-Framework-Experiments/experiments/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
X_train, y_train,X_test, y_test,X_val, y_val=get_data("MotionSense",['accel','gyro'],False)
input_shape=X_train[0].shape
print(input_shape)

(60, 6)


In [3]:
from librep.transforms.simclr import SimCLR
import simclr_utils as sm_ut
transformations=sm_ut.transform_combinations(3,4)

print(transformations)

[[0, 1, 2], [0, 1, 3], [0, 1, 4], [0, 1, 5], [0, 1, 6], [0, 1, 7], [0, 1, 8], [0, 1, 9], [0, 1, 10], [0, 1, 11], [0, 2, 3], [0, 2, 4], [0, 2, 5], [0, 2, 6], [0, 2, 7], [0, 2, 8], [0, 2, 9], [0, 2, 10], [0, 2, 11], [0, 3, 4], [0, 3, 5], [0, 3, 6], [0, 3, 7], [0, 3, 8], [0, 3, 9], [0, 3, 10], [0, 3, 11], [0, 4, 5], [0, 4, 6], [0, 4, 7], [0, 4, 8], [0, 4, 9], [0, 4, 10], [0, 4, 11], [0, 5, 6], [0, 5, 7], [0, 5, 8], [0, 5, 9], [0, 5, 10], [0, 5, 11], [0, 6, 7], [0, 6, 8], [0, 6, 9], [0, 6, 10], [0, 6, 11], [0, 7, 8], [0, 7, 9], [0, 7, 10], [0, 7, 11], [0, 8, 9], [0, 8, 10], [0, 8, 11], [0, 9, 10], [0, 9, 11], [0, 10, 11], [1, 2, 3], [1, 2, 4], [1, 2, 5], [1, 2, 6], [1, 2, 7], [1, 2, 8], [1, 2, 9], [1, 2, 10], [1, 2, 11], [1, 3, 4], [1, 3, 5], [1, 3, 6], [1, 3, 7], [1, 3, 8], [1, 3, 9], [1, 3, 10], [1, 3, 11], [1, 4, 5], [1, 4, 6], [1, 4, 7], [1, 4, 8], [1, 4, 9], [1, 4, 10], [1, 4, 11], [1, 5, 6], [1, 5, 7], [1, 5, 8], [1, 5, 9], [1, 5, 10], [1, 5, 11], [1, 6, 7], [1, 6, 8], [1, 6, 9], [1,

In [ ]:
batch_size_head = 256
decay_steps_head = 1000
epochs_head = 300
temperature_head = 0.77
input_shape=X_train[0].shape
n_components=60
print(input_shape)
#transformations=[['noise', 'channel_shuffle', 'amplify_attenuate']]
verbose=0
import os,torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for transform_funcs in transformations:

    simclr = SimCLR(dataset="motionsense",input_shape=input_shape,n_components=n_components,batch_size=batch_size_head, transform_funcs=transform_funcs,
                                            temperature=temperature_head, epochs=epochs_head,
                                            
                                            verbose=verbose,device=device)
    print(transform_funcs)
    
    trained_simclr_model,epoch_wise_loss = simclr.fit(X_train)


(60, 6)
[0, 1, 2]
exist model
[0, 1, 3]
[0, 1, 4]
[0, 1, 5]
[0, 1, 6]
[0, 1, 7]
[0, 1, 8]
[0, 1, 9]


/home/amparo/unicamp_hyper_2/M4-Framework-Experiments/experiments/.local/lib/python3.10/site-packages/librep/estimators/simclr/torch/dataset_simclr.py:42: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at ../aten/src/ATen/native/Copy.cpp:299.)
  transform_1 = torch.tensor(transform_1, dtype=torch.float32).to(self.device)


[0, 1, 10]
[0, 1, 11]
[0, 2, 3]
[0, 2, 4]
[0, 2, 5]
[0, 2, 6]
[0, 2, 7]
[0, 2, 8]
[0, 2, 9]
[0, 2, 10]
[0, 2, 11]
[0, 3, 4]
[0, 3, 5]
[0, 3, 6]
[0, 3, 7]
[0, 3, 8]
[0, 3, 9]
[0, 3, 10]
[0, 3, 11]
[0, 4, 5]
[0, 4, 6]
[0, 4, 7]
[0, 4, 8]
[0, 4, 9]
[0, 4, 10]
[0, 4, 11]
[0, 5, 6]
[0, 5, 7]
[0, 5, 8]
[0, 5, 9]
[0, 5, 10]
[0, 5, 11]
[0, 6, 7]
[0, 6, 8]
[0, 6, 9]
[0, 6, 10]
[0, 6, 11]
[0, 7, 8]
[0, 7, 9]
[0, 7, 10]
[0, 7, 11]
[0, 8, 9]
[0, 8, 10]
[0, 8, 11]
[0, 9, 10]
[0, 9, 11]
[0, 10, 11]
[1, 2, 3]
[1, 2, 4]
[1, 2, 5]
[1, 2, 6]
[1, 2, 7]
[1, 2, 8]
[1, 2, 9]
[1, 2, 10]
[1, 2, 11]
[1, 3, 4]
[1, 3, 5]
[1, 3, 6]
[1, 3, 7]
[1, 3, 8]
[1, 3, 9]
[1, 3, 10]
[1, 3, 11]
[1, 4, 5]
[1, 4, 6]
[1, 4, 7]
[1, 4, 8]
[1, 4, 9]
[1, 4, 10]
[1, 4, 11]
[1, 5, 6]
[1, 5, 7]
[1, 5, 8]
[1, 5, 9]
[1, 5, 10]
[1, 5, 11]
[1, 6, 7]
[1, 6, 8]
[1, 6, 9]
[1, 6, 10]
[1, 6, 11]
[1, 7, 8]
[1, 7, 9]
[1, 7, 10]
[1, 7, 11]
[1, 8, 9]
[1, 8, 10]
[1, 8, 11]
[1, 9, 10]
[1, 9, 11]
[1, 10, 11]
[2, 3, 4]
[2, 3, 5]
[2, 3, 6]
[2, 3, 7]
[2

In [ ]:
print(simclr.model)

In [ ]:
data=simclr.transform(X_train)
data_t=simclr.transform(X_test)
print(data.shape)
print(X_train.shape)
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=200, random_state=42)  
# Entrenar el clasificador
clf.fit(data, y_train)
etiquetas_pred = clf.predict(data_t)
from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(y_test, etiquetas_pred)
report = classification_report(y_test, etiquetas_pred)
print(accuracy,report)

In [ ]:
data=simclr.predict(X_train)
data_t=simclr.predict(X_test)
print(data.shape)
print(X_train.shape)
from sklearn.ensemble import RandomForestClassifier

# Crear un clasificador Random Forest
clf = RandomForestClassifier(n_estimators=100, random_state=42)  # Puedes ajustar los hiperparámetros según sea necesario

# Entrenar el clasificador
clf.fit(data, y_train)
etiquetas_pred = clf.predict(data_t)
from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(y_test, etiquetas_pred)
report = classification_report(y_test, etiquetas_pred)
print(accuracy,report)